### 1. Load the data

In [4]:
import numpy as np
import pandas as pd
from glob import glob
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
import tensorflow as tf

df= pd.read_csv("../../scene-classification/train.csv")
df = df.replace(0,'buildings').replace(1,'forest').replace(2,'glacier').replace(3,'mountain').replace(4,'sea').replace(5,'street')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

fig = plt.figure(figsize=(20,5))
for i in range(36):
    ax = fig.add_subplot(3, 12, i + 1, xticks=[], yticks=[])
    ax.imshow(np.squeeze(x_train[i]))

### 2.Image Augmentation

In [13]:
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras import preprocessing

datagen = ImageDataGenerator(rescale=1./255,
                           shear_range = 0.2,
                           zoom_range = 0.2,
                           horizontal_flip=True,
                           validation_split=0.25)
train_generator = datagen.flow_from_dataframe(dataframe=df,
                        directory="../../scene-classification/train",
                        x_col="image_name",
                        y_col="label",
                        subset="training",
                        batch_size=100,
                        seed=42,
                        shuffle=True,
                        class_mode="categorical",
                        target_size=(150,150))
valid_generator = datagen.flow_from_dataframe(dataframe=df,
                        directory="../../scene-classification/train",
                        x_col="image_name",
                        y_col="label",
                        subset="validation", batch_size=10,
                        seed=42,
                        shuffle=True,
                        class_mode="categorical",
                        target_size=(150,150))

Found 12776 validated image filenames belonging to 6 classes.
Found 4258 validated image filenames belonging to 6 classes.


### 3.Initial model architecture

In [6]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential()
model.add(Conv2D(filters=16, kernel_size=2, padding='same', activation='relu', 
                        input_shape=(150, 150, 3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(6, activation='softmax'))

model.summary()

W0817 08:32:46.917914 140518171060032 module_wrapper.py:137] From /home/ubuntu/anaconda3/envs/dog-project/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0817 08:32:46.937942 140518171060032 module_wrapper.py:137] From /home/ubuntu/anaconda3/envs/dog-project/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0817 08:32:46.964821 140518171060032 deprecation.py:506] From /home/ubuntu/anaconda3/envs/dog-project/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 150, 150, 16)      208       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 75, 75, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 75, 75, 32)        2080      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 37, 37, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 37, 37, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 43808)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 500)               21904500  
__________

In [7]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9003540044149114064
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 4440791980657115457
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 594241504545580189
physical_device_desc: "device: XLA_GPU device"
]


In [8]:
# import os
# os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

# import tensorflow as tf
# Creates a graph.
a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(c))

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device

[[22. 28.]
 [49. 64.]]


In [9]:
# model.compile(loss='categorical_crossentropy',optimizer = optimizers.RMSprop(lr=1e-4),metrics=['acc'])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

W0817 08:32:51.742069 140518171060032 module_wrapper.py:137] From /home/ubuntu/anaconda3/envs/dog-project/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [10]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

checkpointer = ModelCheckpoint(filepath='model.weights.best.hdf5', verbose=1, 
                               save_best_only=True)

history =model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator, callbacks=[checkpointer],
                    validation_steps=STEP_SIZE_VALID,
                    epochs=20,verbose=1)

W0817 08:32:53.514430 140518171060032 deprecation.py:323] From /home/ubuntu/anaconda3/envs/dog-project/lib/python3.6/site-packages/tensorflow_core/python/ops/math_grad.py:1424: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/20
127/127 [==============================] - 200s 2s/step - loss: 3.0793 - acc: 0.4737 - val_loss: 0.9104 - val_acc: 0.6664
Epoch 2/20
127/127 [==============================] - 196s 2s/step - loss: 0.8703 - acc: 0.6790 - val_loss: 0.8026 - val_acc: 0.6989
Epoch 3/20
127/127 [==============================] - 195s 2s/step - loss: 0.7778 - acc: 0.7148 - val_loss: 0.7730 - val_acc: 0.7114
Epoch 4/20
127/127 [==============================] - 195s 2s/step - loss: 0.6973 - acc: 0.7458 - val_loss: 0.7457 - val_acc: 0.7312
Epoch 5/20
127/127 [==============================] - 194s 2s/step - loss: 0.6680 - acc: 0.7564 - val_loss: 0.7157 - val_acc: 0.7392
Epoch 6/20
127/127 [==============================] - 195s 2s/step - loss: 0.6302 - acc: 0.7764 - val_loss: 0.6403 - val_acc: 0.7669
Epoch 7/20
127/127 [==============================] - 196s 2s/step - loss: 0.5935 - acc: 0.7872 - val_loss: 0.6149 - val_acc: 0.7834
Epoch 8/20
127/127 [==============================] - 195s 2s/step - 

In [11]:
model.save("InitialModel.h5")

### Calculate classification accuracy on test set

In [12]:
model.evaluate_generator(generator=valid_generator, steps=STEP_SIZE_VALID)

[0.5692151790184868, 0.815442557529092]